In [1]:
import torch
from torchvision import datasets, transforms, models
import time

# 定义修改后的ResNet模型
class ModifiedResNet(torch.nn.Module):
    def __init__(self):
        super(ModifiedResNet, self).__init__()
        # 加载ResNet模型，不使用预训练权重，并修改第一层卷积层
        self.resnet = models.resnet18(pretrained=False)  # 使用ResNet18，不使用预训练权重
        self.resnet.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)  # 修改第一层卷积层以适应MNIST的单通道输入
        self.resnet.fc = torch.nn.Linear(self.resnet.fc.in_features, 10)  # 修改最后的全连接层以适应MNIST的10个类别
    
    def forward(self, x):
        x = self.resnet(x)
        return torch.nn.functional.log_softmax(x, dim=1)

# 修改训练函数以适应修改后的ResNet模型
def train_model(workers, epochs, batch_size, target_loss, threshold):
    torch.manual_seed(123)
    
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # ResNet需要224x224的输入尺寸
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    
    train_set = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=workers, drop_last=True)
    
    model = ModifiedResNet()

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    
    start_time = time.time()
    last_loss = float('inf')
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        avg_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch+1} - Loss: {avg_loss}")
        
        if avg_loss <= target_loss or abs(last_loss - avg_loss) < threshold:
            end_time = time.time()
            total_time = end_time - start_time
            return avg_loss, total_time
        
        last_loss = avg_loss
    
    end_time = time.time()
    total_time = end_time - start_time
    
    return avg_loss, total_time

# 超参数网格搜索函数
def grid_search(target_loss=0.0001, threshold=0.00001):
    best_time = float('inf')
    best_hyperparams = {}
    
    for workers in range(1,9):
        epochs = 100
        for batch_size in [64,128]:
            print(f"Workers: {workers}, Epochs: {epochs}, Batch Size: {batch_size}")
            
            loss, time_taken = train_model(workers, epochs, batch_size, target_loss, threshold)
            
            if loss <= target_loss and time_taken < best_time:
                best_time = time_taken
                best_hyperparams = {
                    'Workers': workers,
                    'Epochs': epochs,
                    'Batch Size': batch_size
                }
                
            print(f"Loss: {loss}, Time Taken: {time_taken} seconds\n")
    
    print("最佳超参数:")
    print(best_hyperparams)
    print(f"达到目标损失{target_loss}的时间: {best_time}秒")

# 执行超参数搜索
grid_search(target_loss=0.0001)


Workers: 1, Epochs: 100, Batch Size: 128


/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1 - Loss: 0.9649289050736488
Epoch 2 - Loss: 0.17235254493151975
Epoch 3 - Loss: 0.099612935171582
Epoch 4 - Loss: 0.07281329842586802
Epoch 5 - Loss: 0.05766813257016624
Epoch 6 - Loss: 0.047502378003799126
Epoch 7 - Loss: 0.040233642315595515
Epoch 8 - Loss: 0.03447292428901499
Epoch 9 - Loss: 0.030576497928287167
Epoch 10 - Loss: 0.025983339133791816
Epoch 11 - Loss: 0.023216396827413898
Epoch 12 - Loss: 0.020455345005477563
Epoch 13 - Loss: 0.018165157708383173
Epoch 14 - Loss: 0.016156173731065076
Epoch 15 - Loss: 0.013985179038817843
Epoch 16 - Loss: 0.012527342904140202
Epoch 17 - Loss: 0.011033693633468136
Epoch 18 - Loss: 0.01008767841118149
Epoch 19 - Loss: 0.008818041209855841
Epoch 20 - Loss: 0.0076871571050455365
Epoch 21 - Loss: 0.006785982405830525
Epoch 22 - Loss: 0.00636473605205289
Epoch 23 - Loss: 0.005764107833045304
Epoch 24 - Loss: 0.005274416051599054
Epoch 25 - Loss: 0.004786611617613449
Epoch 26 - Loss: 0.004318386578855789
Epoch 27 - Loss: 0.003827934286